# Packages

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import pandas as pd
import os
import io
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras import mixed_precision

2024-04-09 07:30:23.680263: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 07:30:23.680395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 07:30:23.837788: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print('Device:', tpu.master())
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# except:
#     strategy = tf.distribute.get_strategy()
# print('Number of replicas:', strategy.num_replicas_in_sync)

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128
CLASSES = ['0', '1', '2', '3', '4']
EPOCHS = 25

# Data Loading

In [3]:
paths = []
labels = []

df = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")

for i in df["image_id"]:
    paths.append(i)

for i in df["label"]:
    labels.append(i)


# divide each list for each dataset object


def divide_list(input_list, chunk_size):
    
    return [input_list[i:i + chunk_size] for i in range(0, len(input_list), chunk_size)]

paths = divide_list(paths, 3000)
labels = divide_list(labels, 3000)


val_num = 2000

if len(paths[-1]) > val_num:
    
    excess = len(paths[-1]) - val_num
    
    for i in [paths, labels]:
        for sub in i[-1][:excess]:
            i[-2].append(sub)

        del i[-1][:excess]

        
if len(paths[-1]) < 1000:
    
    delete_elements = 1000 - len(paths[-1])
    
    for i in [paths, labels]:
        
        for sub in i[-2]:
            i[-1].append(sub)
            if len(i[-1]) == 1000:
                break

        del i[-2][:delete_elements]

print([len(path) for path in paths])

print(sum([len(path) for path in paths]))

[3000, 3000, 3000, 3000, 3000, 3000, 2397, 1000]
21397


# No TFRecord Files

In [4]:
# create each dataset object

datasets_no_tfrecord = []

for path, label in zip(paths, labels):
    
    # Create image dataset
    image_dataset = tf.data.Dataset.from_tensor_slices(path)

    # Function to load and preprocess each image
    def load_and_preprocess(subpath):
        image = tf.io.read_file("/kaggle/input/cassava-leaf-disease-classification/train_images/" + subpath)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [512, 512])
        return image

    image_dataset = image_dataset.map(load_and_preprocess)

    # Create dataset for labels
    label_dataset = tf.data.Dataset.from_tensor_slices(label)

    dataset_no_tfrecord = tf.data.Dataset.zip((image_dataset, label_dataset))
    
    datasets_no_tfrecord.append(dataset_no_tfrecord)

train_datasets_no_tfrecord = datasets_no_tfrecord[:-1]
val_datasets_no_tfrecord = datasets_no_tfrecord[-1]


# preprocess

# def augment(image, label):
    
#     # Apply random contrast
#     image = tf.image.random_contrast(image, lower=0.3, upper=0.5)
    
#     # Apply random rotation
#     image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    
#     return image, label



for i in range(len(train_datasets_no_tfrecord)):
    train_datasets_no_tfrecord[i] = train_datasets_no_tfrecord[i].batch(batch_size = BATCH_SIZE).map(lambda x, y: (x/255, y)).shuffle(500).prefetch(buffer_size=AUTOTUNE)

val_datasets_no_tfrecord = val_datasets_no_tfrecord.cache("val_cache").batch(batch_size = BATCH_SIZE).map(lambda x, y: (x/255, y)).shuffle(500).prefetch(buffer_size=AUTOTUNE)

# Model

In [ ]:
# use two gpu's
strategy = tf.distribute.MirroredStrategy()

# mixed-precision training
mixed_precision.set_global_policy('mixed_float16')


with strategy.scope():
    
    model = models.Sequential([layers.Input(shape=(512, 512, 3)),
                               
                               layers.RandomContrast(factor=0.4),
                               layers.RandomRotation(factor=0.3),
                               
                               layers.Conv2D(32, kernel_size=(3, 3)),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3)),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3)),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3)),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Flatten(),
                               layers.Dense(128),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.Dense(5, activation="softmax")])
    
    model.compile(optimizer="adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
for epoch in range(EPOCHS):
    
    for dataset in train_datasets_no_tfrecord:
        
        if train_datasets_no_tfrecord[-1] == dataset: # only use validation data for the last dataset
            history = model.fit(dataset, epochs=1, validation_data=val_datasets_no_tfrecord, verbose = 0)
            print(".", end="")
            
        else:
            history = model.fit(dataset, epochs=1, verbose = 0)
            print(".", end="")
    
    print(f"\nEPOCH {epoch + 1}")
    
    train_loss = history.history['loss']
    train_accuracy = history.history['accuracy']
    val_loss = history.history['val_loss']
    val_accuracy = history.history['val_accuracy']

    print(f"Train Loss: {train_loss[-1]}")
    print(f"Train Accuracy: {train_accuracy[-1]}")
    print(f"Validation Loss: {val_loss[-1]}")
    print(f"Validation Accuracy: {val_accuracy[-1]}")

In [ ]:
scores = model.evaluate(test_dataset)

# Retraining

### Data Loading

In [7]:
# for entire models

import shutil

# Define source and destination paths
src_path = r"/kaggle/input/trained_model1.keras/keras/eight/1/trained_model8_200_epochs_entire.keras"
dst_path = r"/kaggle/working/"

# Copy the file
shutil.copy(src_path, dst_path)

saved_model = tf.keras.models.load_model('/kaggle/working/trained_model8_200_epochs_entire.keras')

saved_weights = saved_model.get_weights()

In [ ]:
# for saved_weights using pkl

import shutil
import pickle

# Define source and destination paths
src_path = r"/kaggle/input/trained_model1.keras/keras/three/1/trained_model3_75_epochs_weights.pkl"
dst_path = r"/kaggle/working/"

# Copy the file
shutil.copy(src_path, dst_path)

# Load the weights with pickle
with open('/kaggle/working/trained_model3_75_epochs_weights.pkl', 'rb') as f:
    loaded_weights = pickle.load(f)

In [8]:
saved_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_contrast_1               │ (None, 512, 512, 3)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_1               │ (None, 512, 512, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 510, 510, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 510, 510, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 510, 510, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 255, 255, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 253, 253, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 124, 124, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 124, 124, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 124, 124, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 2, 2, 64)       │             

 Total params: 712,721 (2.72 MB)

 Trainable params: 237,573 (928.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 475,148 (1.81 MB)

### Model

In [9]:
# use two gpu's
strategy = tf.distribute.MirroredStrategy()

# mixed-precision training
mixed_precision.set_global_policy('mixed_float16')


with strategy.scope():
    
    model = models.Sequential([layers.Input(shape=(512, 512, 3)),
                               
                               layers.RandomContrast(factor=0.4),
                               layers.RandomRotation(factor=0.3),
                               
                               layers.Conv2D(32, kernel_size=(3, 3)),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3)),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3)),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Conv2D(64, kernel_size=(3,3)),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size = (2, 2)),
                               
                               layers.Flatten(),
                               layers.Dense(128),
                               layers.Dropout(0.2),
                               layers.Activation('relu'),
                               layers.Dense(5, activation="softmax")])
    
    model.set_weights(saved_weights)
    
    model.compile(optimizer="adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_contrast                 │ (None, 512, 512, 3)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 512, 512, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 510, 510, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 510, 510, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 510, 510, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 255, 255, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 253, 253, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 124, 124, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 124, 124, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 124, 124, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 2, 2, 64)       │             

 Total params: 237,573 (928.02 KB)

 Trainable params: 237,573 (928.02 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
train_accuracies = []
val_accuracies = []

for epoch in range(EPOCHS):
    
    for dataset in train_datasets_no_tfrecord:
        
        if train_datasets_no_tfrecord[-1] == dataset: # only use validation data for the last dataset
            history = model.fit(dataset, epochs=1, validation_data=val_datasets_no_tfrecord, verbose = 0)
            print(".", end="")
            
        else:
            history = model.fit(dataset, epochs=1, verbose = 0)
            print(".", end="")
    
    print(f"\nEPOCH {epoch + 1}")
    
    train_loss = history.history['loss']
    train_accuracy = history.history['accuracy']
    val_loss = history.history['val_loss']
    val_accuracy = history.history['val_accuracy']
    
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Train Loss: {train_loss[-1]}")
    print(f"Train Accuracy: {train_accuracy[-1]}")
    print(f"Validation Loss: {val_loss[-1]}")
    print(f"Validation Accuracy: {val_accuracy[-1]}")

.......
EPOCH 1
Train Loss: 0.4158637523651123
Train Accuracy: 0.8490408658981323
Validation Loss: 0.4462500810623169
Validation Accuracy: 0.8500000238418579
.......
EPOCH 2
Train Loss: 0.4401680827140808
Train Accuracy: 0.8507089018821716
Validation Loss: 0.47238266468048096
Validation Accuracy: 0.828000009059906
.......
EPOCH 3
Train Loss: 0.42247068881988525
Train Accuracy: 0.8523769974708557
Validation Loss: 0.4492935836315155
Validation Accuracy: 0.8460000157356262
.......
EPOCH 4
Train Loss: 0.37912634015083313
Train Accuracy: 0.8748957514762878
Validation Loss: 0.44316548109054565
Validation Accuracy: 0.8460000157356262
.......
EPOCH 5
Train Loss: 0.41699734330177307
Train Accuracy: 0.854045033454895
Validation Loss: 0.4394077658653259
Validation Accuracy: 0.8500000238418579
.......
EPOCH 6
Train Loss: 0.39556989073753357
Train Accuracy: 0.8640533685684204
Validation Loss: 0.41560766100883484
Validation Accuracy: 0.8579999804496765
.......
EPOCH 7
Train Loss: 0.4192236661911011


In [17]:
model.save("trained_model10_250_epochs_entire.keras")

# you can save the entire model using: tf.saved_model.save(model, filename), model.save(filename)
# you can save the weights using: np.save(), models.save_weights(), or pickle

import pickle

# Get the weights
weights = model.get_weights()

# Save the weights with pickle
with open('/kaggle/working/trained_model3_weights_dataset_objects.pkl', 'wb') as f:
    pickle.dump(weights, f)

In [18]:
train_accuracies = [i[0] for i in train_accuracies]
val_accuracies = [i[0] for i in val_accuracies]

with open("trained_model10_acc_multiple_datasets", "w") as f:
    for acc in train_accuracies:
        f.write(str(acc) + "\n")

with open("trained_model10_val_acc_multiple_datasets", "w") as f:
    for val in val_accuracies:
        f.write(str(val) + "\n")

# Scratch

In [87]:
# pre-trained model
pretrained_model = tf.keras.applications.ResNet50(include_top=False,
                                                  input_shape=(512, 512, 3),
                                                  pooling='avg',
                                                  weights='imagenet')

pretrained_model.trainable = False

In [96]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    
    model = models.Sequential([pretrained_model,
                             tf.keras.layers.Flatten(input_shape=pretrained_model.output_shape[1:]),
                             tf.keras.layers.Dense(1),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Activation('relu'),
                             tf.keras.layers.Dense(5, activation="softmax")])
    
    model.compile(optimizer="adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

model.evaluate(val_datasets_no_tfrecord)

8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.1806 - loss: 3.2596


[1.7026786804199219, 0.09399999678134918]

In [85]:
# for entire models

import shutil

# Define source and destination paths
src_path = r"/kaggle/input/trained_model1.keras/keras/eight/1/trained_model8_200_epochs_entire.keras"
dst_path = r"/kaggle/working/"

# Copy the file
shutil.copy(src_path, dst_path)

saved_model = tf.keras.models.load_model('/kaggle/working/trained_model8_200_epochs_entire.keras')

flatten_layer = saved_model.layers[0].get_weights()
dense_layer = saved_model.layers[-1].get_weights()


model = models.Sequential([tf.keras.layers.Input((2048,)),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(128),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.Activation('relu'),
                           tf.keras.layers.Dense(5, activation="softmax")])

model.layers[0].set_weights(flatten_layer)
model.layers[-1].set_weights(dense_layer)

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model1 = models.Sequential([pretrained_model,
                               model])

    model1.compile(optimizer="adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

model1.evaluate(val_datasets_no_tfrecord)

8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.1972 - loss: 4.4394


[2.320650815963745, 0.10599999874830246]

In [82]:
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_14 (Flatten)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_37 (Activation)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 262,917 (1.00 MB)

 Trainable params: 262,917 (1.00 MB)

 Non-trainable params: 0 (0.00 B)

In [80]:
print(model.layers[0])

<Flatten name=flatten_13, built=True>


In [75]:
count = 0

for w in saved_weights:
    if w == []:
        count += 1

print(count)

3
